# 创建vLLM服务器

需要使用vllm部署一个OpenAI API接口的服务器

# 执行测评

使用`EvalScope`命令进行评测，直接在终端输入以下命令

```bash
evalscope eval \
  --model Qwen3-8B \
  --api-url http://localhost:8000/v1 \
  --api-key EMPTY \
  --eval-type service \
  --eval-batch-size 16 \
  --datasets iquiz \
  --work-dir outputs/Qwen3-8B
```

# 使用OpenCompass评测后端

查看支持的数据集列表

In [1]:
from evalscope.backend.opencompass import OpenCompassBackendManager
OpenCompassBackendManager.list_datasets()

/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 06-03 06:51:41 [__init__.py:243] Automatically detected platform cuda.


2025-06-03 06:51:43,100	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


['ocnli_fc',
 'RTE',
 'triviaqa',
 'GaokaoBench',
 'chid',
 'cluewsc',
 'humaneval',
 'mmlu',
 'BoolQ',
 'ARC_e',
 'C3',
 'math',
 'siqa',
 'Xsum',
 'cmb',
 'CMRC',
 'ARC_c',
 'bustm',
 'WSC',
 'lambada',
 'storycloze',
 'eprstmt',
 'AX_g',
 'ocnli',
 'mbpp',
 'piqa',
 'DRCD',
 'lcsts',
 'obqa',
 'CB',
 'cmnli',
 'cmmlu',
 'MultiRC',
 'hellaswag',
 'afqmc',
 'gsm8k',
 'agieval',
 'COPA',
 'race',
 'WiC',
 'AX_b',
 'ReCoRD',
 'csl',
 'summedits',
 'ceval',
 'tnews',
 'winogrande',
 'PMMEval',
 'strategyqa',
 'nq']

支持从 ModelScope 自动下载数据集，要启用此功能，请设置环境变量：
```bash
export DATASET_SOURCE=ModelScope
```

配置文件
参数说明:
- eval_backend：默认值为 OpenCompass，表示使用 OpenCompass 评测后端
- eval_config：字典，包含以下字段：
    - datasets：列表，参考目前支持的数据集
    - models：字典列表，每个字典必须包含以下字段：
        - path：OpenAI API 请求模型名称。
            - 若使用 vLLM 或 LMDeploy 部署模型，则设置为对应的 model ID；
        - openai_api_base：OpenAI API 的URL。
        - is_chat：布尔值，设置为 True 表示聊天模型，设置为 False 表示基础模型。
        - key：模型 API 的 OpenAI API 密钥，默认值为 EMPTY。
    - work_dir：字符串，保存评测结果、日志和摘要的目录。默认值为 outputs/default。
    - limit： 可以是 int、float 或 str，例如 5、5.0 或 '[10:20]'。默认值为 None，表示运行所有示例。


In [17]:
task_cfg_dict = dict(
    eval_backend='OpenCompass',
    eval_config={
        'datasets': ["mmlu", "ceval",'ARC_c', 'gsm8k'],
        'models': [
            {'path': 'Qwen3-8B', 
            'openai_api_base': 'http://127.0.0.1:8000/v1/chat/completions', 
            'is_chat': True,
            'batch_size': 16
            }
        ],
        'work_dir': 'outputs/qwen3_eval_result',
        'limit': 10,
        },
    )

In [19]:
from evalscope.run import run_task
from evalscope.summarizer import Summarizer

def run_eval():
    task_cfg = task_cfg_dict
    run_task(task_cfg=task_cfg)

    print('>> Start to get the report with summarizer ...')
    report_list = Summarizer.get_report_from_cfg(task_cfg)
    print(f'\n>> The report list: {report_list}')

run_eval()

2025-06-03 07:38:10,258 - evalscope - INFO - Args: Task config is provided with dictionary type.
2025-06-03 07:38:10,262 - evalscope - INFO - Check the OpenCompass environment: OK
2025-06-03 07:38:10,263 - evalscope - INFO - Dump task config to ./outputs/20250603_073810/configs/task_config_51dccf.yaml
2025-06-03 07:38:10,266 - evalscope - INFO - {
    "model": "DummyCustomModel",
    "model_id": "dummy-model",
    "model_args": {},
    "model_task": "text_generation",
    "template_type": null,
    "chat_template": null,
    "datasets": [],
    "dataset_args": {},
    "dataset_dir": "/root/.cache/modelscope/hub/datasets",
    "dataset_hub": "modelscope",
    "generation_config": {},
    "eval_type": "custom",
    "eval_backend": "OpenCompass",
    "eval_config": {
        "datasets": [
            "mmlu",
            "ceval",
            "ARC_c",
            "gsm8k"
        ],
        "models": [
            {
                "path": "Qwen3-8B",
                "openai_api_base": "http

06/03 07:38:13 - OpenCompass - INFO - Current exp folder: outputs/qwen3_eval_result/20250603_073810
06/03 07:38:20 - OpenCompass - INFO - Partitioned into 111 tasks.


launch OpenICLInfer[Qwen3-8B/ARC-c] on CPU 


                                               
  5%|▌         | 6/111 [01:10<12:47,  7.31s/it]

launch OpenICLInfer[Qwen3-8B/ceval-computer_network] on CPU 
launch OpenICLInfer[Qwen3-8B/ceval-operating_system] on CPU 


...

launch OpenICLEval[Qwen3-8B/lukaemon_mmlu_conceptual_physics] on CPU 


100%|██████████| 111/111 [04:51<00:00,  2.62s/it]

06/03 08:02:29 - OpenCompass - ERROR - /usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/opencompass/runners/base.py - summarize - 64 - OpenICLEval[Qwen3-8B/ceval-computer_network] failed with code 1
06/03 08:02:29 - OpenCompass - ERROR - /usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/opencompass/runners/base.py - summarize - 64 - OpenICLEval[Qwen3-8B/ceval-operating_system] failed with code 1
06/03 08:02:29 - OpenCompass - ERROR - /usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/opencompass/runners/base.py - summarize - 64 - OpenICLEval[Qwen3-8B/ceval-computer_architecture] failed with code 1
06/03 08:02:29 - OpenCompass - ERROR - /usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/opencompass/runners/base.py - summarize - 64 - OpenICLEval[Qwen3-8B/ceval-college_programming] failed with code 1
06/03 08:02:29 - OpenCompass - ERROR - /usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/opencompass/runners/base.py - summarize - 6


2025-06-03 08:02:29,224 - evalscope - INFO - Args: Task config is provided with dictionary type.
2025-06-03 08:02:29,224 - evalscope - INFO - **Loading task cfg for summarizer: {
    "model": "DummyCustomModel",
    "model_id": "dummy-model",
    "model_args": {},
    "model_task": "text_generation",
    "template_type": null,
    "chat_template": null,
    "datasets": [],
    "dataset_args": {},
    "dataset_dir": "/root/.cache/modelscope/hub/datasets",
    "dataset_hub": "modelscope",
    "generation_config": {},
    "eval_type": "custom",
    "eval_backend": "OpenCompass",
    "eval_config": {
        "datasets": [
            "mmlu",
            "ceval",
            "ARC_c",
            "gsm8k"
        ],
        "models": [
            {
                "path": "Qwen3-8B",
                "openai_api_base": "http://127.0.0.1:8000/v1/chat/completions",
                "is_chat": true,
                "batch_size": 16,
                "abbr": "Qwen3-8B"
            }
        ],
   

>> Start to get the report with summarizer ...

>> The report list: [{'dataset': 'ARC-c', 'version': '1e0de5', 'metric': 'accuracy', 'mode': 'gen', 'Qwen3-8B': '20.00'}, {'dataset': 'ceval-computer_network', 'version': '-', 'metric': '-', 'mode': '-', 'Qwen3-8B': '-'}, {'dataset': 'ceval-operating_system', 'version': '-', 'metric': '-', 'mode': '-', 'Qwen3-8B': '-'}, {'dataset': 'ceval-computer_architecture', 'version': '-', 'metric': '-', 'mode': '-', 'Qwen3-8B': '-'}, {'dataset': 'ceval-college_programming', 'version': '-', 'metric': '-', 'mode': '-', 'Qwen3-8B': '-'}, {'dataset': 'ceval-college_physics', 'version': '-', 'metric': '-', 'mode': '-', 'Qwen3-8B': '-'}, {'dataset': 'ceval-college_chemistry', 'version': '-', 'metric': '-', 'mode': '-', 'Qwen3-8B': '-'}, {'dataset': 'ceval-advanced_mathematics', 'version': '-', 'metric': '-', 'mode': '-', 'Qwen3-8B': '-'}, {'dataset': 'ceval-probability_and_statistics', 'version': '-', 'metric': '-', 'mode': '-', 'Qwen3-8B': '-'}, {'datase